In [1]:
from libs.alibaba.alibaba import Alibaba
from libs.alibaba.p4p import P4P
from libs.json import JSON

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import redis
import pendulum
import arrow
import imp
import time
import re

import types
import traceback 
import threading

In [2]:
broker_url = 'redis://localhost:6379/0'
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
p4p = P4P(market, market['lid'], market['lpwd'],broker_url=broker_url, headless=False)

. storage markets.json
.\storage\markets.json was deserialized!
E:\glitter\Eyelashes_config [] p4p_keywords_list_recording.json
E:\glitter\Eyelashes_config\p4p_keywords_list_recording.json was deserialized!
E:\glitter\Eyelashes_config [] p4p_keywords_list_monitor.json
E:\glitter\Eyelashes_config\p4p_keywords_list_monitor.json was deserialized!


In [12]:
def crawl(self, group="all"):

    try:
        keywords = []
        # self.load_keywords('recording')
        with self.lock:
            self.load_url()
            # all_kws_count = int(self.browser.find_element_by_css_selector('a.all-kwcount span').text)
            all_kws_count = self.switch_to_group(group)

            kws_count = 0
            while True:
                css_selector = "div.keyword-manage .bp-table-main-wraper>table"
                table = self.browser.find_element_by_css_selector(css_selector)

                trs = table.find_elements_by_css_selector('tbody tr')

                has_checked = False
                for idx, tr in enumerate(trs):
                    kws_count += 1

                    print('index:', idx, len(trs), end=' > ')
                    ActionChains(self.browser).move_to_element(tr).perform()
                    id = tr.find_element_by_css_selector('td:first-child input').get_attribute('value').strip()
                    # if id not in self.keywords_list['recording']:
                    #     print('skipped_not_in_recording')
                    #     continue

                    grp = tr.find_element_by_css_selector('td:nth-child(4)').text.strip()
                    if group != 'all':
                        if group not in grp:
                            print('skipped_not_in_group', group, grp)
                            continue

                    dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    kws = tr.find_element_by_css_selector('td:nth-child(3)').text.strip()

                    print(kws, end=' > ')

                    if not self.open_price_dialog(tr):
                        print('skipped_2')
                        continue

                    [prices, sponsors] = self.find_prices_and_sponsors()

                    if prices:
                        keywords.append([dt, id, kws, grp, prices, sponsors])

                        if (id not in self.recent_prices):
                            self.recent_prices[id] = []
                        self.recent_prices[id].append(prices)
                        if len(self.recent_prices[id]) > 5:
                            self.recent_prices[id].pop(0)

                    print(' >>>>>> successful end ')

                if not self.next_page():
                    break
        if keywords:
            print('==================================')
            self.save_crawling_result(keywords)

    except Exception as e:
        print('Error: ', e)
        traceback.print_exc()

p4p.crawl = types.MethodType(crawl, p4p)

In [10]:
def save_crawling_result(self, keywords):
    root = self.market['directory'] + '_config'
    date_str = keywords[0][0].split(' ')[0]
    fn = 'p4p_keywords_crawl_result_'+date_str+'.json.gz'
    print(fn, root)
    JSON.serialize(keywords, root, [], fn, append=True)
    
p4p.save_crawling_result = types.MethodType(save_crawling_result, p4p)

In [20]:
def find_prices_and_sponsors(self, close=True):
    prices = []
    sponsors = None

    kws = self.browser.find_element_by_css_selector(
        '.sc-manage-edit-price-dialog span[data-role="span-keyword"]').text.strip()
    sponsors = self.find_sponsors(kws)

    while True:
        try:
            price_table_tbody_selector = ".sc-manage-edit-price-dialog table tbody,.sc-manage-edit-price-dialog p.util-clearfix"
            price_table_tbody = WebDriverWait(self.browser, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, price_table_tbody_selector)))

            if price_table_tbody.tag_name == 'p':
                prices = None
                break

            for btn in price_table_tbody.find_elements_by_css_selector('a'):
                price = btn.text.strip()
                float(price)
                prices.append(price)

            self.check_balance()
            break
        except StaleElementReferenceException:
            #             self.browser.implicitly_wait(0.5)
            time.sleep(0.5)
            continue
        except ValueError:
            prices = []
            break
    if close:
        webdriver.ActionChains(self.browser).send_keys(Keys.ESCAPE).perform()
    return [prices, sponsors]
p4p.find_prices_and_sponsors = types.MethodType(find_prices_and_sponsors, p4p)

In [13]:
def check_balance(self):
    try:
        balance = self.browser.find_element_by_css_selector(
            '.sc-manage-edit-price-dialog span[data-role="span-balance"]').text

        if self.broker_url:
            r = redis.StrictRedis.from_url(self.broker_url)
            self.balance = r.getset(self.market['name']+'_p4p_balance', balance)
            
            if float(balance) == self.initial_balance or self.balance is None:
                return
            else:
                self.balance = self.balance.decode()
                print(self.initial_balance, self.balance, balance, balance != self.balance)
                if self.balance != balance:
                    diff = format(float(self.balance) - float(balance), '.2f')

                    time_str = arrow.now().format('YYYY-MM-DD HH:mm:ss')
                    date_str = time_str.split(' ')[0]
                    root = self.market['directory'] + '_config'
                    fn = 'p4p_balance_change_history_' + date_str + '.json.gz'
                    JSON.serialize([time_str, diff], root, [], fn, append=True)
        else:
            if self.balance is None:
                self.balance = balance
            elif float(balance) == self.initial_balance:
                return
            elif self.balance != balance:
                diff = format(float(self.balance) - float(balance), '.2f')
                self.balance = balance

                time_str = arrow.now().format('YYYY-MM-DD HH:mm:ss')
                date_str = time_str.split(' ')[0]
                root = self.market['directory'] + '_config'
                fn = 'p4p_balance_change_history_'+date_str+'.json.gz'
                JSON.serialize([time_str, diff], root, [], fn, append=True)
    except Exception as e:
        print(e)
        traceback.print_exc()
p4p.check_balance = types.MethodType(check_balance, p4p)

In [ ]:
p4p.browser.set_window_size(1920, 1200)

In [ ]:
p4p.browser.save_screenshot('screenie.png')

In [4]:
group = '关注词'

In [ ]:
p4p.load_url()
p4p.switch_to_group(group)

In [15]:
p4p.crawl(group=group)

index: 0 7 > 3d eyelash extension > 300 18.61 68.61 True
 >>>>>> successful end 
index: 1 7 > 3d faux mink eyelashes > 300 68.61 68.61 False
 >>>>>> successful end 
index: 2 7 > 3d mink eyelashes > 300 68.61 68.61 False
 >>>>>> successful end 
index: 3 7 > 3d silk eyelash > 300 68.61 68.61 False
 >>>>>> successful end 
index: 4 7 > eyelash extension > 300 68.61 68.61 False
 >>>>>> successful end 
index: 5 7 > eyelash extension factory > 300 68.61 68.61 False
 >>>>>> successful end 
index: 6 7 > mink eyelashes > 300 68.61 68.61 False
 >>>>>> successful end 
next_page: False


In [6]:
p4p.monitor(group=group)

Task Monitor start to run with group="关注词"
switch to group: 关注词
index: 0 7 > 3d eyelash extension > make selected >  >>>>>> successful end 
index: 1 7 > 3d faux mink eyelashes > make selected >  >>>>>> successful end 
index: 2 7 > 3d mink eyelashes > make selected >  >>>>>> successful end 
index: 3 7 > 3d silk eyelash > make selected >  >>>>>> successful end 
index: 4 7 > eyelash extension > make selected >  >>>>>> successful end 
index: 5 7 > eyelash extension factory > make selected >  >>>>>> successful end 
index: 6 7 > mink eyelashes > make selected >  >>>>>> successful end 
switch selected on.
next_page: False


In [7]:
p4p.turn_all_off(group=group)

next_page: False


In [29]:
float('300.00') == 300

True

In [ ]:
p4p.kws_tracking

In [ ]:
# p4p.load_url()

# css_selector = "div.keyword-manage .bp-table-main-wraper>table"
# table = p4p.browser.find_element_by_css_selector(css_selector)
# trs = table.find_elements_by_css_selector('tbody tr')
# idx = 1
# for tr in trs:
#     print()
#     print(idx, end=': ')
#     idx += 1
    
#     kws = tr.find_element_by_css_selector('td:nth-child(3)').text.strip()
#     print(kws, end=": ")
        
#     if not p4p.open_price_dialog(tr):
#         print('skipped_2')
#         idx += 1
#         continue
        
#     [prices, sponsors] = p4p.find_prices_and_sponsors()
#     print(prices, sponsors)
#     if idx > 10:
#         break

In [ ]:
tr = trs[8]
kws = tr.find_element_by_css_selector('td:nth-child(3)').text.strip()
print(kws, end=": ")

if not p4p.open_price_dialog(tr):
    print('skipped_2')
else:
#     time.sleep(1)
    prices = p4p.find_prices()

In [ ]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Tools']

In [ ]:
p4p.find_sponsor_list_position(kws='strip eyelash')

In [ ]:
group = '直通车App'

In [ ]:
p4p.monitor(group=group)

In [ ]:
p4p.kws_tracking = {}

In [ ]:
p4p.turn_all_off(group=group)

In [ ]:
text = "adsf"

In [ ]:
a,b = text.split(' ')